In [1]:
import os
try:
    print(file_path)
except:
    file_path = os.path.abspath('')
    os.chdir(os.path.dirname(file_path))
    print(file_path)

/root/code_Bao/News_mongo_redis/notebook


In [12]:
import sys
sys.path.append("")

from datetime import datetime
import requests
from dotenv import load_dotenv
load_dotenv()
import os

from src.crawl_and_summarize_news import *

In [3]:
news_scraper = NewsScraper()
formatted_docs = news_scraper.get_all_news()

Crawling url: https://znews.vn/
Crawling url: https://cafef.vn/


In [4]:
print(formatted_docs[2][0].page_content)
print(formatted_docs[2][0].metadata)

Năm 2023 phá kỷ lục về lượng tài liệu lưu hành trên thư viện số. Hà Lan là một câu chuyện rõ ràng cho thành công của việc đưa người dân đến với thế giới văn học số.

Nhiều quan chức của nền tảng hàng đầu về cung cấp sách điện tử cho thư viện OverDrive cho biết năm 2023 là một năm phá kỷ lục về lượng lưu hành trên thư viện số, với số lượng người từng sử dụng trên thư viện số tăng 19% so với năm 2022.

Dấu mốc phát triển thư viện số năm 2023

Tổng cộng, người dùng thư viện trên toàn thế giới đã mượn khoảng 662 triệu sách điện tử, sách nói số và tạp chí số, đại diện OverDrive công bố. Ngoài ra, 152 hệ thống thư viện đã báo cáo hơn một triệu lượt thanh toán số vào năm 2023, tăng so với 129 lượt vào năm ngoái.

Lượng người dùng các nền tảng thư viện số liên tục tăng. Ảnh: Goodereader.

Ứng dụng đọc sách số của OverDrive, Libby, cũng đã đạt được một cột mốc quan trọng vào đầu năm 2023, vượt qua một tỷ lượt dùng kể từ khi ra mắt. Số lượt cài đặt ứng dụng này cũng đã tăng 22% trong cùng năm.



In [9]:
full_news = news_scraper.take_news_elements(news_url='https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html')

In [11]:
full_news[0].page_content

'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.\n\nKhông còn xuất hiện trên web Apple hay các thị trường lớn, iPhone 11, iPhone 12 vẫn được bán tại Việt Nam. Ảnh: PhoneArena.\n\nKhi ra mắt dòng iPhone 15 hồi tháng 9/2023, Apple âm thầm loại bỏ iPhone 12 khỏi danh sách thiết bị hãng còn sản xuất và kinh doanh. Mẫu iPhone 11 cũng bị “khai tử” trước đó một năm, cả hai không còn hiện diện trên cửa hàng trực tuyến của hãng. Tuy nhiên, hai chiếc máy nói trên vẫn đang được bán rộng rãi tại thị trường Việt Nam thông các kênh bán lẻ.\n\nTrong đó, chiếc iPhone 11 hiện có giá niêm yết khoảng 10 triệu đồng, không có nhiều chênh lệch giữa các đại lý. Phiên bản 64 GB của model nói trên là chiếc iPhone chính hãng giá rẻ nhất đang còn bán tại Việt Nam. Cao hơn khoảng 3 triệu đồng là chiếc iPhone 12 bộ nhớ mặc định. Cả hai mẫu máy này đều không có mặt trên trang web chính thức của Apple.\n\nMẫu máy 5 năm tuổi vẫn được sản xu

In [10]:
full_news[0].metadata

{'title': "Bộ đôi iPhone mãi vẫn không bị 'khai tử' ở Việt Nam",
 'link': 'https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html',
 'authors': ['Xuân Sang'],
 'language': 'vi',
 'description': 'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.',
 'publish_date': datetime.datetime(2024, 2, 2, 18, 46, 52, tzinfo=tzoffset(None, 25200))}

In [14]:
input_dict = full_news[0].metadata
full_news = {
    'title': input_dict['title'],
    'link': input_dict['link'],
    'authors': input_dict['authors'],
    'publish_date': input_dict['publish_date'],
    'page_content': full_news[0].page_content,
}

In [ ]:

class NewsSummarizer:
    def __init__(self, summarizer = pipeline("summarization", 
                                             model="Falconsai/text_summarization", device = take_device()),
                 translator = GoogleTranslator(),
                 max_length:int=230, 
                 min_length:int=30,
                 ):
        self.summarizer = summarizer
        self.translator = translator
        self.max_length = max_length
        self.min_length = min_length
        
    def summary_text(self,text:str)->str:
        '''Summary short text'''
        sum_text = self.summarizer(text, max_length=self.max_length, 
                                   min_length=self.min_length, do_sample=False)[0]['summary_text']
        return sum_text
    
    def summary_news(self, news:str, chunk_overlap:str = 0)->str:
        with no_ssl_verification():
            text_splitter = TokenTextSplitter(chunk_size=self.max_length * 2,
                                            chunk_overlap=chunk_overlap)
            
            trans_news = self.translator.translate(text=news, to_lang='en')
            text_chunks = text_splitter.split_text(trans_news)
            summary_documents = [self.summary_text(chunk) for chunk in text_chunks]
            summary_text = '\n'.join(summary_documents)

            summary_text = self.translator.translate(text=summary_text, to_lang='vi')
        return summary_text
    
    def summary_news_for_redis(self, full_news:dict, content_element:str = 'page_content'):
        '''
        full_news is like
        {'title': 'Lì xì hơn 6.000 cuốn sách tại Đường sách Tết 2024',
            'link': 'https://znews.vn/li-xi-hon-6-nghin-cuon-sach-tai-duong-sach-tet-2024-post1457886.html',
            'authors': ['Đông Miên'],
            'publish_date': datetime.datetime(2024, 1, 31, 7, 0, 41, tzinfo=tzoffset(None, 25200)),
            'page_content': 'Ban tổ chức}

        Output is with the summary text
        '''

        text = full_news['page_content']
        full_news['summary_text'] = self.summary_news(news = text)
        return full_news


In [15]:
new_summarizer = NewsSummarizer()
sum_news = new_summarizer.summary_news_for_redis(full_news= full_news)

In [17]:
sum_news

{'title': "Bộ đôi iPhone mãi vẫn không bị 'khai tử' ở Việt Nam",
 'link': 'https://znews.vn/bo-doi-iphone-mai-van-khong-bi-khai-tu-o-viet-nam-post1458086.html',
 'authors': ['Xuân Sang'],
 'publish_date': datetime.datetime(2024, 2, 2, 18, 46, 52, tzinfo=tzoffset(None, 25200)),
 'page_content': 'iPhone 11, iPhone 12 không còn trên website Apple, nhưng vẫn được sản xuất mới để bán ở những thị trường như Việt Nam, Ấn Độ.\n\nKhông còn xuất hiện trên web Apple hay các thị trường lớn, iPhone 11, iPhone 12 vẫn được bán tại Việt Nam. Ảnh: PhoneArena.\n\nKhi ra mắt dòng iPhone 15 hồi tháng 9/2023, Apple âm thầm loại bỏ iPhone 12 khỏi danh sách thiết bị hãng còn sản xuất và kinh doanh. Mẫu iPhone 11 cũng bị “khai tử” trước đó một năm, cả hai không còn hiện diện trên cửa hàng trực tuyến của hãng. Tuy nhiên, hai chiếc máy nói trên vẫn đang được bán rộng rãi tại thị trường Việt Nam thông các kênh bán lẻ.\n\nTrong đó, chiếc iPhone 11 hiện có giá niêm yết khoảng 10 triệu đồng, không có nhiều chênh lệ